In [1]:
# import warnings

# warnings.simplefilter(action="ignore", category=FutureWarning)
# warnings.simplefilter(action="ignore", category=UserWarning)

In [2]:
import os
from collections import Counter
from math import ceil

import numpy as np
import torch
from fastembed import TextEmbedding
from gait import FEL, FEL1, FEL2, Layers
from rich.pretty import pprint

In [3]:
layers_json = "~/data/NorthSea.json"
layers = Layers.load(layers_json)

In [4]:
base, ext = os.path.splitext(layers_json)
base

'~/data/NorthSea'

In [5]:
fel = FEL(layers)

In [6]:
pprint(fel.create_line_0(), expand_all=True)

FELLine(
│   line='Locate pipelines with current phase ends with service and destination facility that match the pattern 36/22-bp',
│   fel=FEL1(
│   │   layer='Pipelines',
│   │   where="current_phase LIKE '%SERVICE' and to_facility LIKE '%36/22-BP%'"
│   )
)

In [7]:
func = fel.create_line_0

In [8]:
elems = [func() for _ in range(100)]
elems = {_.line: _.fel for _ in elems}

In [9]:
docs = list(elems.keys())
fels = list(elems.values())

In [10]:
len(docs)

100

In [11]:
# for _ in random.choices(docs, k=3):
#     print(_)
#     print()

In [42]:
for _ in TextEmbedding.list_supported_models():
    print(_["model"])

BAAI/bge-base-en
BAAI/bge-base-en-v1.5
BAAI/bge-large-en-v1.5
BAAI/bge-small-en
BAAI/bge-small-en-v1.5
BAAI/bge-small-zh-v1.5
sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
thenlper/gte-large
mixedbread-ai/mxbai-embed-large-v1
snowflake/snowflake-arctic-embed-xs
snowflake/snowflake-arctic-embed-s
snowflake/snowflake-arctic-embed-m
snowflake/snowflake-arctic-embed-m-long
snowflake/snowflake-arctic-embed-l
jinaai/jina-clip-v1
intfloat/multilingual-e5-large
sentence-transformers/paraphrase-multilingual-mpnet-base-v2
Qdrant/clip-ViT-B-32-text
sentence-transformers/all-MiniLM-L6-v2
jinaai/jina-embeddings-v2-base-en
jinaai/jina-embeddings-v2-small-en
jinaai/jina-embeddings-v2-base-de
jinaai/jina-embeddings-v2-base-code
jinaai/jina-embeddings-v2-base-zh
jinaai/jina-embeddings-v2-base-es
thenlper/gte-base
nomic-ai/nomic-embed-text-v1.5
nomic-ai/nomic-embed-text-v1.5-Q
nomic-ai/nomic-embed-text-v1


In [45]:
# https://qdrant.github.io/fastembed/examples/Supported_Models/
model = TextEmbedding(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
)

In [46]:
embeddings: list[np.ndarray] = list(
    model.query_embed(docs)
)  # notice that we are casting the generator to a list

In [47]:
np.savez(os.path.expanduser(f"{base}.npz"), embeddings=embeddings)

In [48]:
prompt = func().line
print(prompt)
embedding = list(model.query_embed(prompt))[0]

similarities = np.dot(embeddings, embedding)

delta = similarities.max() - similarities.min()
top = ceil(5 + 15 * delta)
delta, top

Identify pipelines with current operator is different from conocophillips skandinavia as


(np.float32(0.60486317), 15)

In [49]:
# top = 15

In [51]:
lyr_cnt = Counter()
fel_cnt = Counter()
opr_cnt = Counter()

for index in similarities.argsort()[-top:]:
    print(docs[index])
    fel = fels[index]
    if isinstance(fel, FEL2):
        lyr_cnt[fel.layer1] += 1
        lyr_cnt[fel.layer2] += 1
        opr_cnt[fel.relation] += 1
        fel_cnt["fel2"] += 1
    if isinstance(fel, FEL1):
        lyr_cnt[fel.layer] += 1
        fel_cnt["fel1"] += 1

print(fel_cnt.most_common(2))
print(lyr_cnt.most_common(3))
print(opr_cnt.most_common(3))

Find all pipelines that are within a distance of 6.7 feet of pipelines
List all pipelines where country is no and where destination facility is different from frigg tp1-y that are within 91.3 miles of pipelines
List all pipelines with current phase is abandoned in place and originating facility is unspecified or empty that are inside discoveries
Identify all pipelines where originating facility ends with heimdal and medium is empty
Identify all pipelines that are within 4.8 meters of wellbores where has oil samples = 0
Show pipelines with current phase starting with decommissioned that intersect discoveries where field name is empty
Identify pipelines where dimension between 36.0 and 26.0
Locate pipelines that intersect pipelines
Show all discoveries where discovery type below -1 that contain pipelines with current operator is unspecified and destination facility is h-7 bp
List all pipelines that intersect pipelines
Show all pipelines with originating facility beginning with ormen and 